# Merging SOC 2018 and Census 2010 Occupational Codes



## Statement of the Problem

To explore the question of flexibility, the inherent properties of occupations should be considered. To do this, I use data from O\*Net to build an index of flexibility for a particular occupation. However, the O\*Net database uses the Standard Occupation Classification (SOC) Code, updated annually, while the CPS and ATUS use the 2010 Census Occupation Classification Codes. 

Linking the occupation codes requires utilizing the Crosswalk (https://www2.census.gov/programs-surveys/demo/guidance/industry-occupation/2018-occupation-code-list-and-crosswalk.xlsx). The design of the Crosswalk between 2018 SOC and 2010 Census is not friendly to easy linking. 

In order to use the Crosswalk to link the 2018 SOC with the 2010 Census, we must first extract some codes from the 2018 Census Title column, fill in empty cells (imported as NaN), create a dictionary, and append the 2010 Census codes to some O\*Net data. 

With the final .csv, we can merge the CPS/ATUS data with the O\*Net job characteristics via the 2010 Census occupation code. 

## Extracting 2018 SOC Codes

In [1]:
import pandas as pd

Download the crosswalk table, removing the first three rows of title, date updated, and white space. 

Looking at the first ten rows of the data, we see that there are SOC Codes in the Census Title column. We want to extract these codes and put them into the SOC Code column. 

In [2]:
crosswalk = pd.read_excel('https://www2.census.gov/programs-surveys/demo/guidance/industry-occupation/2018-occupation-code-list-and-crosswalk.xlsx', 
                sheet_name='2010 to 2018 Crosswalk ',
                header=3)

crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers
2,11-1031,0030,Legislators,11-1031,30.0,Legislators
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers
8,NaN,NaN,NaN,NaN,NaN,Public Relations Managers (11-2032)
9,NaN,NaN,NaN,NaN,NaN,Fundraising Managers (11-2033)


To extract the codes, we are going to use the split function. First we split along the first parenthesis, then we split along the second parenthesis, and drop the unnecessary columns.

In [3]:
crosswalk[['2018 Census Title short','tmp_Code']] = crosswalk['2018 Census Title '].str.split("(", expand=True)

crosswalk[['Code','paren']] = crosswalk['tmp_Code'].str.split(")", expand=True)

crosswalk.drop(['tmp_Code', 'paren'], axis = 1, inplace = True)

crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,NaN,NaN,NaN,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,NaN,NaN,NaN,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


Now that we have extracted the SOC Codes from the Census title, we need to move the values to the 2018 SOC Code column

In [4]:
crosswalk['2018 SOC Code'].fillna(value=crosswalk['Code'], inplace=True)

crosswalk.head(10)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,NaN,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,NaN,NaN,11-2032,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,NaN,NaN,11-2033,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


Now, we want to forward fill the 2010 Census code to remove the NaN caused by blanks in the Excel spreadsheet.

In [5]:
crosswalk['2010 Census Code'].fillna(method = 'ffill', inplace = True)

crosswalk.head(15)

,2010 SOC code,2010 Census Code,2010 Census Title \n,2018 SOC Code,2018 Census Code,2018 Census Title,2018 Census Title short,Code
0,11-1011,0010,Chief Executives,11-1011,10.0,Chief Executives,Chief Executives,None
1,11-1021,0020,General and Operations Managers,11-1021,20.0,General and Operations Managers,General and Operations Managers,None
2,11-1031,0030,Legislators,11-1031,30.0,Legislators,Legislators,None
3,11-2011,0040,Advertising and Promotions Managers,11-2011,40.0,Advertising and Promotions Managers,Advertising and Promotions Managers,None
4,11-2020,0050,Marketing and Sales Managers,NaN,NaN,NaN,NaN,NaN
5,NaN,0050,NaN,11-2021,51.0,Marketing Managers,Marketing Managers,None
6,NaN,0050,NaN,11-2022,52.0,Sales Managers,Sales Managers,None
7,11-2031,0060,Public Relations and Fundraising Managers,11-2030,60.0,Public Relations and Fundraising Managers,Public Relations and Fundraising Managers,None
8,NaN,0060,NaN,11-2032,NaN,Public Relations Managers (11-2032),Public Relations Managers,11-2032
9,NaN,0060,NaN,11-2033,NaN,Fundraising Managers (11-2033),Fundraising Managers,11-2033


## Dictionary to Match Census and SOC Codes

We are ready to create the dictionary. As we have many SOC codes to one Census code, we want to groupby SOC code and pull the relevant Census code.

In [6]:
crosswalk_dict = dict(crosswalk.groupby('2018 SOC Code')['2010 Census Code'].apply(list))

## Using Dictionary to Rename Occupation Codes

ONet to determine level of flexibility by occupation. Measures include: 
- Work Context — Freedom to Make Decisions (https://www.onetonline.org/find/descriptor/result/4.C.3.a.4)
    - No freedom 0 ~ 100 A lot of freedom
- Work Context — Structured versus Unstructured Work (https://www.onetonline.org/find/descriptor/result/4.C.3.b.8)
    - Structured (no freedom) 0 ~ Unstructured (a lot of freedom)
- Work Context — Time Pressure (https://www.onetonline.org/find/descriptor/result/4.C.3.d.1)
    - Never 0 ~ 100 Every day
- Work Context — Regular Work Schedules (https://www.onetonline.org/find/descriptor/result/4.C.3.d.4)
    - Regular/established schedule 0 ~ 100 Seasonal/only during certain times of the year
- Work Styles — Independence (https://www.onetonline.org/find/descriptor/result/1.C.6)
    - No independence 0 ~ 100 A lot of independence
    - "Job requires developing one's own ways of doing things, guiding oneself with little or no supervision, and depending on oneself to get things done."

These were determined to impact flexibility from the browse tool (https://www.onetonline.org/find/descriptor/browse/)

In [7]:
free = pd.read_csv('../Freedom_to_Make_Decisions.csv')
struct = pd.read_csv('../Structured_versus_Unstructured_Work.csv')
time = pd.read_csv('../Time_Pressure.csv')
sched = pd.read_csv('../Work_Schedules.csv')
indep = free = pd.read_csv('../Freedom_to_Make_Decisions.csv')

In [8]:
free = free.rename(columns = {'Context':'Freedom_to_Make_Decisions'})
struct = struct.rename(columns = {'Context':'Structured_v_Unstructured'})
time = time.rename(columns = {'Context':'Time_Pressure'})
sched = sched.rename(columns = {'Context':'Regular_Schedule'})
indep = indep.rename(columns = {'Context':'Independence'})

In [9]:
tmp = pd.merge(free, struct, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, time, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, sched, how='left', on=['Code','Occupation'])
tmp = pd.merge(tmp, indep, how='left', on=['Code','Occupation'])

onet = tmp
onet.rename(columns = {'Code': '2018 SOC Code'}, inplace=True) # for data straight from ONET
onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence
0,100,29-1213.00,Dermatologists,94,70,0,100
1,100,29-1215.00,Family Medicine Physicians,87,93,0,100
2,100,23-1023.00,"Judges, Magistrate Judges, and Magistrates",98,93,0,100
3,100,29-1024.00,Prosthodontists,93,77,6,100
4,100,29-1229.06,Sports Medicine Physicians,94,72,27,100


In [10]:
onet[['2018 SOC Code','drop']] = onet['2018 SOC Code'].str.split(".", expand=True)
onet.drop(columns='drop', inplace=True)

onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence
0,100,29-1213,Dermatologists,94,70,0,100
1,100,29-1215,Family Medicine Physicians,87,93,0,100
2,100,23-1023,"Judges, Magistrate Judges, and Magistrates",98,93,0,100
3,100,29-1024,Prosthodontists,93,77,6,100
4,100,29-1229,Sports Medicine Physicians,94,72,27,100


Trying to remove the 2010 Census code from the list in the DateFrame showed that there were missing values in the Census Code column. Investigation found that there were two values not mapping in the dictionary. 

- 2018 SOC Code 53-6031 is listed as 56-6031 in the Crosswalk file, but I am confident that this is a typo as it is listed under 53-6030. The corresponding 2010 Census code is 9360.
- 2018 SOC Code 17-3012 is listed as having a space in the Crosswalk file that was not removed in the string split procedure above. The corresponding 2010 Census code is 1540.


In [11]:
onet['2010 Census Code'] = onet['2018 SOC Code'].map(crosswalk_dict)

onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence,2010 Census Code
0,100,29-1213,Dermatologists,94,70,0,100,[3060]
1,100,29-1215,Family Medicine Physicians,87,93,0,100,[3060]
2,100,23-1023,"Judges, Magistrate Judges, and Magistrates",98,93,0,100,[2110]
3,100,29-1024,Prosthodontists,93,77,6,100,[3010]
4,100,29-1229,Sports Medicine Physicians,94,72,27,100,[3060]


In [12]:
nulls = onet[onet['2010 Census Code'].isna()]

nulls

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence,2010 Census Code
235,85,53-6031,Automotive and Watercraft Service Attendants,69,90,11,85,NaN
667,70,17-3012,Electrical and Electronics Drafters,70,83,15,70,NaN


In [13]:
CORRECTION1 = (onet['2018 SOC Code'] == '53-6031')
CORRECTION2 = (onet['2018 SOC Code'] == '17-3012')

onet.loc[CORRECTION1, '2010 Census Code'] = ['9360']
onet.loc[CORRECTION2, '2010 Census Code'] = ['1540']

onet.head()

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence,2010 Census Code
0,100,29-1213,Dermatologists,94,70,0,100,[3060]
1,100,29-1215,Family Medicine Physicians,87,93,0,100,[3060]
2,100,23-1023,"Judges, Magistrate Judges, and Magistrates",98,93,0,100,[2110]
3,100,29-1024,Prosthodontists,93,77,6,100,[3010]
4,100,29-1229,Sports Medicine Physicians,94,72,27,100,[3060]


In [14]:
def remove_from_list(x):
    val = x[0]
    if isinstance(x, str):
        return [x][0]
    return val

In [15]:
onet['2010 Census Code']=onet['2010 Census Code'].apply(remove_from_list)

onet['2010 Census Code'].astype(int)

0      3060
1      3060
2      2110
3      3010
4      3060
       ... 
868    8940
869    7810
870    8410
871    4250
872    4400
Name: 2010 Census Code, Length: 873, dtype: int64

In [16]:
onet

,Freedom_to_Make_Decisions,2018 SOC Code,Occupation,Structured_v_Unstructured,Time_Pressure,Regular_Schedule,Independence,2010 Census Code
0,100,29-1213,Dermatologists,94,70,0,100,3060
1,100,29-1215,Family Medicine Physicians,87,93,0,100,3060
2,100,23-1023,"Judges, Magistrate Judges, and Magistrates",98,93,0,100,2110
3,100,29-1024,Prosthodontists,93,77,6,100,3010
4,100,29-1229,Sports Medicine Physicians,94,72,27,100,3060
...,...,...,...,...,...,...,...,...
868,42,51-9197,Tire Builders,36,79,28,42,8940
869,40,51-3022,"Meat, Poultry, and Fish Cutters and Trimmers",47,90,2,40,7810
870,40,51-6063,"Textile Knitting and Weaving Machine Setters, ...",44,49,15,40,8410
871,38,37-3011,Landscaping and Groundskeeping Workers,54,57,27,38,4250


In [17]:
onet.to_csv('onet_flex.csv', index=False)

## Dingel and Neiman 2020

The data generated by Dingel and Neiman 2020 uses the ONET Release 24.2. This release using O\*Net-SOC 2010 taxonomy, that must be crosswalked to O\*Net-SOC 2019 then O\*Net-SOC 2018 to connect with the crosswalk created above using 2010 Census Codes. 

In [18]:
onet = pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv')

onet.rename(columns = {'onetsoccode': 'O*NET-SOC 2010 Code'}, inplace=True)
onet.head(10) # 968 occupations

,O*NET-SOC 2010 Code,title,teleworkable
0,11-1011.00,Chief Executives,1
1,11-1011.03,Chief Sustainability Officers,1
2,11-1021.00,General and Operations Managers,1
3,11-2011.00,Advertising and Promotions Managers,1
4,11-2021.00,Marketing Managers,1
5,11-2022.00,Sales Managers,1
6,11-2031.00,Public Relations and Fundraising Managers,1
7,11-3011.00,Administrative Services Managers,1
8,11-3021.00,Computer and Information Systems Managers,1
9,11-3031.01,Treasurers and Controllers,1


In [19]:
crosswalk_onet_2019_to_2010 = pd.read_csv('https://www.onetcenter.org/taxonomy/2019/walk/2010_to_2019_Crosswalk.csv?fmt=csv')

crosswalk_onet_2019_to_2010.head(10) # 1164 occupations

,O*NET-SOC 2010 Code,O*NET-SOC 2010 Title,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title
0,11-1011.00,Chief Executives,11-1011.00,Chief Executives
1,11-1011.03,Chief Sustainability Officers,11-1011.03,Chief Sustainability Officers
2,11-1021.00,General and Operations Managers,11-1021.00,General and Operations Managers
3,11-1031.00,Legislators,11-1031.00,Legislators
4,11-2011.00,Advertising and Promotions Managers,11-2011.00,Advertising and Promotions Managers
5,11-2011.01,Green Marketers,11-2011.00,Advertising and Promotions Managers
6,11-2021.00,Marketing Managers,11-2021.00,Marketing Managers
7,11-2022.00,Sales Managers,11-2022.00,Sales Managers
8,11-2031.00,Public Relations and Fundraising Managers,11-2032.00,Public Relations Managers
9,11-2031.00,Public Relations and Fundraising Managers,11-2033.00,Fundraising Managers


In [20]:
crosswalk_onet_2019_to_2018 = pd.read_csv('https://www.onetcenter.org/taxonomy/2019/soc/2019_to_SOC_Crosswalk.csv?fmt=csv')

crosswalk_onet_2019_to_2018.head(10) # 1016 occupations

,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title,2018 SOC Code,2018 SOC Title
0,11-1011.00,Chief Executives,11-1011,Chief Executives
1,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives
2,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers
3,11-1031.00,Legislators,11-1031,Legislators
4,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers
5,11-2021.00,Marketing Managers,11-2021,Marketing Managers
6,11-2022.00,Sales Managers,11-2022,Sales Managers
7,11-2032.00,Public Relations Managers,11-2032,Public Relations Managers
8,11-2033.00,Fundraising Managers,11-2033,Fundraising Managers
9,11-3012.00,Administrative Services Managers,11-3012,Administrative Services Managers


In [33]:
tmp = pd.merge(crosswalk_onet_2019_to_2010, crosswalk_onet_2019_to_2018, how='left', on=['O*NET-SOC 2019 Code', 'O*NET-SOC 2019 Title'])
onetDN = pd.merge(onet, tmp, how = 'left', on=['O*NET-SOC 2010 Code'])

onetDN.head()

,O*NET-SOC 2010 Code,title,teleworkable,O*NET-SOC 2010 Title,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title,2018 SOC Code,2018 SOC Title
0,11-1011.00,Chief Executives,1,Chief Executives,11-1011.00,Chief Executives,11-1011,Chief Executives
1,11-1011.03,Chief Sustainability Officers,1,Chief Sustainability Officers,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives
2,11-1021.00,General and Operations Managers,1,General and Operations Managers,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers
3,11-2011.00,Advertising and Promotions Managers,1,Advertising and Promotions Managers,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers
4,11-2021.00,Marketing Managers,1,Marketing Managers,11-2021.00,Marketing Managers,11-2021,Marketing Managers


In [34]:
onetDN['2010 Census Code'] = onetDN['2018 SOC Code'].map(crosswalk_dict)

In [35]:
# Same null values as before, so we use the same two corrections.

CORRECTION1 = (onetDN['2018 SOC Code'] == '53-6031')
CORRECTION2 = (onetDN['2018 SOC Code'] == '17-3012')

onetDN.loc[CORRECTION1, '2010 Census Code'] = '9360'
onetDN.loc[CORRECTION2, '2010 Census Code'] = '1540'

onetDN.head()

,O*NET-SOC 2010 Code,title,teleworkable,O*NET-SOC 2010 Title,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title,2018 SOC Code,2018 SOC Title,2010 Census Code
0,11-1011.00,Chief Executives,1,Chief Executives,11-1011.00,Chief Executives,11-1011,Chief Executives,[0010]
1,11-1011.03,Chief Sustainability Officers,1,Chief Sustainability Officers,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives,[0010]
2,11-1021.00,General and Operations Managers,1,General and Operations Managers,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers,[0020]
3,11-2011.00,Advertising and Promotions Managers,1,Advertising and Promotions Managers,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers,[0040]
4,11-2021.00,Marketing Managers,1,Marketing Managers,11-2021.00,Marketing Managers,11-2021,Marketing Managers,[0050]


In [36]:
onetDN['2010 Census Code']=onetDN['2010 Census Code'].apply(remove_from_list)

onetDN['2010 Census Code'].astype(int)

0        10
1        10
2        20
3        40
4        50
       ... 
989    9650
990    9650
991    9720
992    6910
993    9740
Name: 2010 Census Code, Length: 994, dtype: int64

In [37]:
onetDN

,O*NET-SOC 2010 Code,title,teleworkable,O*NET-SOC 2010 Title,O*NET-SOC 2019 Code,O*NET-SOC 2019 Title,2018 SOC Code,2018 SOC Title,2010 Census Code
0,11-1011.00,Chief Executives,1,Chief Executives,11-1011.00,Chief Executives,11-1011,Chief Executives,0010
1,11-1011.03,Chief Sustainability Officers,1,Chief Sustainability Officers,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives,0010
2,11-1021.00,General and Operations Managers,1,General and Operations Managers,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers,0020
3,11-2011.00,Advertising and Promotions Managers,1,Advertising and Promotions Managers,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers,0040
4,11-2021.00,Marketing Managers,1,Marketing Managers,11-2021.00,Marketing Managers,11-2021,Marketing Managers,0050
...,...,...,...,...,...,...,...,...,...
989,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0,"Pump Operators, Except Wellhead Pumpers",53-7072.00,"Pump Operators, Except Wellhead Pumpers",53-7072,"Pump Operators, Except Wellhead Pumpers",9650
990,53-7073.00,Wellhead Pumpers,0,Wellhead Pumpers,53-7073.00,Wellhead Pumpers,53-7073,Wellhead Pumpers,9650
991,53-7081.00,Refuse and Recyclable Material Collectors,0,Refuse and Recyclable Material Collectors,53-7081.00,Refuse and Recyclable Material Collectors,53-7081,Refuse and Recyclable Material Collectors,9720
992,53-7111.00,Mine Shuttle Car Operators,0,Mine Shuttle Car Operators,47-5044.00,"Loading and Moving Machine Operators, Undergro...",47-5044,"Loading and Moving Machine Operators, Undergro...",6910


In [38]:
onetDN.to_csv('DingelNeiman2020.csv', index=False)